# 保存和加载模型（）

## 为什么重要？
你花了 3 个小时训练好了一个模型，如果不保存，关掉程序就全没了。保存之后，下次直接加载，不用重新训练。

## 保存模型
```python
torch.save(net.state_dict(),'model.params')
```
逐个拆解：
- `net.state_dict()`  → 把模型的所有参数打包成一个字典。
- `torch.save(字典, '文件名')`  → 保存到硬盘上。

`'model.params'` 是文件名，你可以随意命名，比如叫 `'my_model.pt'`、`'best.pth'` 等。

## 加载模型
```python
# 第1步：先造一个一模一样结构的空网络
net2 = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

# 第2步：把保存的参数灌进去
net2.load_state_dict(torch.load('model.params'))
```
逐个拆解：
- **第1步**：为什么要先造空网络？因为保存的只是参数（权重和偏置的数值），不包含网络结构。所以你要先造一个结构一样的壳子，再把参数灌进去。
- **第2步**：
    - `torch.load('model.params')`    → 从文件读出参数字典。
    - `net2.load_state_dict(字典)`     → 把参数灌进 `net2`。

比喻：
保存 = 把一栋楼的家具清单记下来（参数），存到 U 盘。
加载 = 先盖一栋一模一样的空楼（结构），再按清单把家具搬进去。

## 为什么不直接保存整个模型？
你可能想：为什么不 `torch.save(net,'model')` 直接保存整个网络？

技术上可以，但不推荐！
原因：
- 直接保存整个模型 → 和你的代码绑定，换个环境可能加载失败。
- 只保存参数       → 更通用，更安全，是业界标准做法。

## 完整的保存加载流程
```python
# ===== 训练阶段 =====

# 1. 定义网络
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

# 2. 训练...（省略训练过程）

# 3. 训练完了，保存参数
torch.save(net.state_dict(),'my_model.pth')
print("模型保存好了！")


# ===== 之后要用的时候 =====

# 4. 造一个结构一样的空网络
net2 = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

# 5. 把参数加载进去
net2.load_state_dict(torch.load('my_model.pth'))
print("模型加载好了！可以直接用了！")

# 6. 直接用，不需要重新训练
output = net2(X)
```

## GPU 使用（一分钟搞定）
### 核心就一行
```python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
```
翻译：
如果你电脑有 GPU → 用 GPU；如果没有 → 用 CPU。把结果存到 `device` 这个变量里。

### 然后把东西搬到 GPU 上
```python
net = net.to(device)       # 把模型搬到 GPU
X = X.to(device)           # 把数据搬到 GPU
```
就像搬家：模型和数据必须在同一个地方（都在 GPU 或都在 CPU），否则报错。

### 完整模板
```python
# 1. 选设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. 模型搬过去
net = net.to(device)

# 3. 每次训练时，数据也搬过去
X = X.to(device)
y = y.to(device)

# 4. 正常训练
output = net(X)
```

## 总结：你只需要记住这些
### 保存模型（两行）
```python
# 训练完保存
torch.save(net.state_dict(), '文件名.pth')
```

### 加载模型（三行）
```python
# 造空壳 → 灌参数
net2 = nn.Sequential(...)     # 和之前结构一样
net2.load_state_dict(torch.load('文件名.pth'))
```

### GPU 使用（三行）
```python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)
X = X.to(device)
```